In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import timedelta, date
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows',200)

In [3]:
# start = date(2017, 6, 1)
# end = date(2018, 6, 1)


# df = pd.DataFrame()

# for sdate in range((end - start).days):
#     fdate = (start + timedelta(sdate)).strftime('%y%m%d')
#     url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_" + fdate + ".txt"
#     try:
#         data = pd.read_csv(url)
#         data['file'] = fdate
#         df = df.append(data)
#     except:
#         pass

In [4]:
# df['date'] = pd.to_datetime(df.DATE)

In [5]:
# df['turnid'] = df['STATION'] + '/' + df['SCP'] + '/' + df['C/A']

In [6]:
# pickle.dump(df, open('turnstiledata','wb'))

In [7]:
df = pickle.load(open('turnstiledata','rb'))

In [8]:
#STATION and Line for unique
df['STATION']= df['STATION'] +'/' +df['LINENAME'] 

In [9]:
#group irregular intervals
conditions = [  df.TIME.isin(['23:00:00','00:00:00','01:00:00','02:00:00']),
                df.TIME.isin(['03:00:00','04:00:00','05:00:00','06:00:00']),
                df.TIME.isin(['07:00:00','08:00:00','09:00:00','10:00:00']),
                df.TIME.isin(['11:00:00','12:00:00','13:00:00','14:00:00']),
                df.TIME.isin(['15:00:00','16:00:00','17:00:00','18:00:00']),
                df.TIME.isin(['19:00:00','20:00:00','21:00:00','22:00:00'])]

choices = ['00:00','04:00','08:00','12:00','16:00','20:00']

df['TIME'] = np.select(conditions, choices, default=df.TIME)

In [10]:
#group turnsystles and add day of week
df = df.groupby(['STATION','turnid','date','TIME'], as_index=False).sum()
df['dayofweek'] = df.date.dt.day_name()



In [11]:
#cumulative to descrete
df['entries'] = df.ENTRIES.shift(-1) - df.ENTRIES
df['exits'] = df.EXITS.shift(-1) - df.EXITS
df['passages'] = df.entries + df.exits



In [12]:
#add weekday and weekend
df['weekend/weekday'] = np.where(df['dayofweek'].isin( ['Saturday','Sunday']), 'Weekend','Weekday')
df_weekend= df[df['weekend/weekday']=='Weekend']
df_weekday= df[df['weekend/weekday']!='Weekend']

In [14]:
df_weekend.sample(5)

,STATION,turnid,date,TIME,ENTRIES,EXITS,dayofweek,entries,exits,passages,weekend/weekday
4356377,BERGEN ST/23,BERGEN ST/01-06-01/R618,2018-03-18,04:00,625150,2299096,Sunday,1.0,0.0,1.0,Weekend
8656968,ST. GEORGE/1,ST. GEORGE/01-00-03/S101A,2018-01-14,08:00,2296977,352,Sunday,13.0,0.0,13.0,Weekend
6211706,FULTON ST/2345ACJZ,FULTON ST/00-07-02/N098,2017-12-30,16:00,254971,110210,Saturday,0.0,0.0,0.0,Weekend
1867704,36 ST/DNR,36 ST/00-00-05/C018,2017-05-28,08:00,7056597,1491576,Sunday,1071.0,65.0,1136.0,Weekend
2913589,66 ST-LINCOLN/1,66 ST-LINCOLN/01-00-03/R159,2017-10-07,08:00,7542357,6624468,Saturday,125.0,113.0,238.0,Weekend


In [15]:
#get the median value per station per time block and count how many times
df_weekend_median = df_weekend.groupby(['STATION','TIME'], as_index=False).median()

df_weekend_count = df_weekend.groupby(['STATION','TIME'], as_index=False).count()
df_weekend_median['count'] = df_weekend_count.iloc[:,4]

df_weekend_median = df_weekend_median.drop(columns =['ENTRIES','EXITS'])

sorted_df_weekend=df_weekend_median[df_weekend_median['count']>4].sort_values(by='passages', ascending = False)



In [16]:
sorted_df_weekend.sample(5)

,STATION,TIME,entries,exits,passages,count
800,7 AV/FG,08:00,141.0,45.0,192.0,1047
1399,CARROLL ST/FG,04:00,8.0,4.0,14.0,1137
2601,TIMES SQ-42 ST/ACENQRS1237W,00:00,14.0,13.0,34.0,817
1104,AVENUE X/F,00:00,0.0,0.0,0.0,302
2516,SMITH-9 ST/FG,04:00,10.5,7.5,19.0,622


In [ ]:
df_weekday_median = df_weekday.groupby(['STATION','TIME'], as_index=False).median()

df_weekday_count = df_weekday.groupby(['STATION','TIME'], as_index=False).count()
df_weekday_median['count'] = df_weekday_count.iloc[:,4]

df_weekday_median = df_weekday_median.drop(columns =['ENTRIES','EXITS'])


In [ ]:
sorted_df_weekday=df_weekday_median[df_weekday_median['count']>15].sort_values(by='passages', ascending = False)
sorted_df_weekday

In [ ]:
# build tourist index

In [ ]:
weekday_8am = df_weekday_median[df_weekday_median.TIME == '08:00'].sort_values(by='STATION')
weekday_8am = weekday_8am[weekday_8am.passages >0]
weekday_8am


In [ ]:
weekend_noon =df_weekend_median[df_weekend_median.TIME =='12:00'].sort_values(by='STATION')
weekend_noon =weekend_noon.drop(columns=['TIME','entries','exits','count'])
weekend_noon =weekend_noon[weekend_noon.passages>0]
weekend_noon

In [ ]:
tourist_index = weekday_8am[:]

In [ ]:
tourist_index = tourist_index.merge(weekend_noon, how= 'inner', left_on='STATION', right_on='STATION', suffixes= ['_weekday','_weekend'])

In [ ]:
tourist_index['tourist'] = (tourist_index.passages_weekend/tourist_index.passages_weekday)**.5

In [ ]:
tourist_index.sort_values(by='tourist')

In [ ]:
plt.hist((tourist_index.tourist))

In [ ]:
no_index_station = pd.DataFrame(list(set(df_weekday_median.STATION).difference( set(tourist_index.STATION))))
no_index_station['tourist'] = tourist_index.tourist.median()
no_index_station = no_index_station.rename(columns= {0:'STATION'})

In [ ]:
no_index_station

In [ ]:
tourist_index = tourist_index[['STATION','tourist']].append(no_index_station).reset_index(drop=True)

In [ ]:
tourist_index

In [ ]:
sorted_df_weekday = sorted_df_weekday.merge(tourist_index, how = 'inner', left_on= 'STATION', right_on= 'STATION')

In [ ]:
sorted_df_weekday['passages_local'] = sorted_df_weekday['passages'] / sorted_df_weekday['tourist']


In [ ]:
sorted_df_weekend= sorted_df_weekend.merge(tourist_index, how = 'inner', left_on ='STATION', right_on='STATION')

In [ ]:
sorted_df_weekend['passages_local'] = sorted_df_weekend.passages / sorted_df_weekend.tourist


In [ ]:
top_weekend=sorted_df_weekend.groupby('STATION', as_index = False).mean().sort_values(by = 'passages_local',ascending = False)

In [ ]:
top_weekend=top_weekend[0:6]['STATION']
top_weekend

In [ ]:
top_weekday=sorted_df_weekday.groupby('STATION', as_index= False).mean().sort_values(by='passages_local',ascending =False)

In [ ]:
top_weekday=top_weekday[0:6]['STATION']

In [ ]:
top_weekday

In [ ]:
weekend_to_plot = sorted_df_weekend[sorted_df_weekend['STATION'].isin(top_weekend)]
weekend_to_plot = weekend_to_plot[weekend_to_plot.TIME!='00:00']
weekend_to_plot = weekend_to_plot[weekend_to_plot.TIME!='04:00']
weekend_to_plot = weekend_to_plot[weekend_to_plot.TIME!='20:00']
weekday_to_plot = sorted_df_weekday[sorted_df_weekday['STATION'].isin(top_weekday)]
weekday_to_plot = weekday_to_plot[weekday_to_plot.TIME!='00:00']
weekday_to_plot = weekday_to_plot[weekday_to_plot.TIME!='04:00']
weekday_to_plot = weekday_to_plot[weekday_to_plot.TIME!='20:00']

In [ ]:
ax = plt.axes()
sns.heatmap(weekend_to_plot.pivot('STATION','TIME','passages_local'),xticklabels =['8AM','12PM','4PM'],linewidths=1,ax =ax,
           cmap='coolwarm')
ax.set_title('Weekend Subway Foot Traffic Local')
plt.savefig('weekendSubwayFootTraffic.svg',bbox_inches="tight")
plt.show();




In [ ]:
ax = plt.axes()
sns.heatmap(weekday_to_plot.pivot('STATION','TIME','passages_local'),xticklabels =['8AM','12PM','4PM'],linewidths=1,
           cmap ='coolwarm',ax=ax)
ax.set_title('Weekday Subway Foot Traffic Local')
plt.savefig('weekdaySubwayFootTraffic.svg',bbox_inches="tight")
plt.show;